In [22]:
#auto reload 
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
import pandas as pd
import numpy as np
import os 
from datasets import load_dataset, load_from_disk
import torch 

In [24]:
import pandas as pd 
 
bow_english = pd.read_csv("data/unigram_freq_en.csv")
bow_french = pd.read_csv("data/unigram_freq_fr.csv")

In [25]:
train_len, val_len = 10000, 1000

In [26]:
wmt14 = load_dataset("wmt14", "fr-en", data_dir="data/")

# Accessing example data
train_data = wmt14["train"]
val_data = wmt14["validation"]

if train_len is not None:
    train_data = train_data.select(range(train_len))
if val_len is not None:
    val_data = val_data.select(range(val_len))


In [27]:
examples = 4 
#check some data and their translation
for i, t in zip(range(examples), train_data):
    ex = t["translation"]
    print(ex["fr"], " -> ", ex["en"], "\n\n")



Reprise de la session  ->  Resumption of the session 


Je déclare reprise la session du Parlement européen qui avait été interrompue le vendredi 17 décembre dernier et je vous renouvelle tous mes vux en espérant que vous avez passé de bonnes vacances.  ->  I declare resumed the session of the European Parliament adjourned on Friday 17 December 1999, and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period. 


Comme vous avez pu le constater, le grand "bogue de l'an 2000" ne s'est pas produit. En revanche, les citoyens d'un certain nombre de nos pays ont été victimes de catastrophes naturelles qui ont vraiment été terribles.  ->  Although, as you will have seen, the dreaded 'millennium bug' failed to materialise, still the people in a number of countries suffered a series of natural disasters that truly were dreadful. 


Vous avez souhaité un débat à ce sujet dans les prochains jours, au cours de cette période de session.  ->  You 

In [28]:
from sacremoses import MosesTokenizer, MosesDetokenizer

# Sample sentence
sentence = "This is an  example sentence with words that may not be in the shortlist."

mt_en = MosesTokenizer(lang="en")
mt_fr = MosesTokenizer(lang="fr")

tokenized = mt_en.tokenize(sentence)
tokenized


['This',
 'is',
 'an',
 'example',
 'sentence',
 'with',
 'words',
 'that',
 'may',
 'not',
 'be',
 'in',
 'the',
 'shortlist',
 '.']

In [29]:
class TokenizerWrapper:
    def __init__(self, tokenizer_en, tokenizer_fr):
        self.tokenizer_en = tokenizer_en
        self.tokenizer_fr = tokenizer_fr
    
    def tokenize_function(self, examples):
        return {"tokenized_en" : self.tokenizer_en.tokenize(examples["translation"]["en"]),
                "tokenized_fr" : self.tokenizer_fr.tokenize(examples["translation"]["fr"])}
    
tokenizer_wrapper = TokenizerWrapper(mt_en, mt_fr)


In [30]:
if not os.path.exists("processed_data/tokenized_train_data{}".format(train_len)):
    tokenized_train_data  = train_data.map(tokenizer_wrapper.tokenize_function, batched=False, num_proc=19, remove_columns=["translation"])
    tokenized_train_data.save_to_disk("processed_data/tokenized_train_data{}".format(train_len))


In [31]:
    
if not os.path.exists("processed_data/tokenized_val_data{}".format(val_len)):
    tokenized_val_data  = val_data.map(tokenizer_wrapper.tokenize_function, batched=False, num_proc=19, remove_columns=["translation"])
    tokenized_val_data.save_to_disk("processed_data/tokenized_val_data{}".format(val_len))


Saving the dataset (1/1 shards): 100%|██████████| 1000/1000 [00:00<00:00, 75041.67 examples/s]


In [32]:
tokenized_train_data = load_from_disk("processed_data/tokenized_train_data{}".format(train_len))
tokenized_val_data = load_from_disk("processed_data/tokenized_val_data{}".format(val_len))

In [12]:
# import pandas as pd
# df = pd.read_csv("data/Copy of Lexique383.csv")
# df.sort_values(by="9_freqfilms2", ascending=False, inplace=True)
# only_words_freq = df[["1_ortho", "9_freqfilms2"]][:30000]
# #drop index
# only_words_freq.reset_index(drop=True, inplace=True)
# #change column names
# only_words_freq.columns = ["word", "count"]
# only_words_freq.to_csv("data/unigram_freq_fr.csv", index=False)

In [33]:
most_frequent_english_words = bow_english["word"][:30000].apply(lambda x: str(x)).tolist()
most_frequent_english_words.append("UNK")
most_frequent_french_words = bow_french["word"][:30000].apply(lambda x: str(x)).tolist()
most_frequent_french_words.append("UNK")
tokenized_most_frequent_english_words = mt_en.tokenize(" ".join(most_frequent_english_words))
tokenized_most_frequent_french_words = mt_fr.tokenize(" ".join(most_frequent_french_words))

In [34]:
class toIdTransform:
    def __init__(self, most_frequent_words, tor):
        self.most_frequent_words = most_frequent_words
        self.word_to_id = {word: i for i, word in enumerate(most_frequent_words)}
        self.tor = tor
    
    def __call__(self, tokenized):
        return {"ids_en":[self.word_to_id.get(token, self.word_to_id["UNK"]) for token in tokenized["tokenized_en"]],
                "ids_fr":[self.word_to_id.get(token, self.word_to_id["UNK"]) for token in tokenized["tokenized_fr"]]}
    
to_id_transform = toIdTransform(tokenized_most_frequent_english_words, torch)


In [35]:
if not os.path.exists("processed_data/id_train_data{}".format(train_len)):
    tokenized_train_data = tokenized_train_data.map(to_id_transform, batched=False, num_proc=19)
    tokenized_train_data.save_to_disk("processed_data/id_train_data{}".format(train_len))


Saving the dataset (1/1 shards): 100%|██████████| 10000/10000 [00:00<00:00, 315814.74 examples/s]


In [36]:
if not os.path.exists("processed_data/id_val_data{}".format(val_len)):
    tokenized_val_data = tokenized_val_data.map(to_id_transform, batched=False, num_proc=19)
    tokenized_val_data.save_to_disk("processed_data/id_val_data{}".format(val_len))

Saving the dataset (1/1 shards): 100%|██████████| 1000/1000 [00:00<00:00, 113672.94 examples/s]


In [38]:
tokenized_train_data = load_from_disk("processed_data/id_train_data{}".format(train_len))
tokenized_val_data = load_from_disk("processed_data/id_val_data{}".format(val_len))



Dataset({
    features: ['tokenized_en', 'tokenized_fr', 'ids_en', 'ids_fr'],
    num_rows: 10000
})


In [39]:
class to_tensor:
    def __init__(self, tor):
        self.tor = tor
    
    def __call__(self, ids):
        return {"ids_en":self.tor.tensor(ids["ids_en"]),
                "ids_fr":self.tor.tensor(ids["ids_fr"])}
    
tokenized_train_data = tokenized_train_data.map(to_tensor(torch), batched=False, num_proc=19)
tokenized_val_data = tokenized_val_data.map(to_tensor(torch), batched=False, num_proc=19)

Map (num_proc=19): 100%|██████████| 1000/1000 [00:00<00:00, 3469.57 examples/s]


In [40]:
def to_vector(ids, vocab_size):
    vecs = torch.zeros(len(ids), vocab_size)
    vecs[range(len(ids)), ids] = 1
    return vecs

class to_vec:
    def __init__(self, tor, vocab_size, lang):
        self.tor = tor
        self.vocab_size = vocab_size
        self.lang = lang

    
    def __call__(self, ids):
        ids = self.tor.tensor(ids["ids_{}".format(self.lang)])
        vecs = self.tor.zeros(len(ids), self.vocab_size)
        vecs[range(len(ids)), ids] = 1
        return vecs
    

In [27]:
# to_vec_transform_en = to_vec(torch, len(tokenized_most_frequent_english_words), "en")
# to_vec_transform_fr = to_vec(torch, len(tokenized_most_frequent_french_words), "fr")
# vecs_en = [

# ]
# vecs_fr = [

# ]
# for i in range(len(tokenized_train_data)):
#     vecs_en.append(to_vec_transform_en(tokenized_train_data[i]))
#     vecs_fr.append(to_vec_transform_fr(tokenized_train_data[i]))



In [41]:
from models.fcnn import FCNN

model = FCNN(len(tokenized_most_frequent_english_words), [10,10], 30000, "cuda" if torch.cuda.is_available() else "cpu",torch.nn.ReLU())

In [43]:
for i in range(len(tokenized_train_data)):

    tensorized = torch.tensor(tokenized_train_data[i]["ids_en"]).to("cuda" if torch.cuda.is_available() else "cpu")
    
    result = model(to_vector(tensorized, len(tokenized_most_frequent_english_words)).to("cuda" if torch.cuda.is_available() else "cpu"))
    
    print(result)
    
    break

tensor([[-0.2523,  0.2291, -0.2632,  ..., -0.0914,  0.2034,  0.0489],
        [-0.2531,  0.2293, -0.2637,  ..., -0.0914,  0.2036,  0.0495],
        [-0.2523,  0.2294, -0.2630,  ..., -0.0919,  0.2035,  0.0494],
        [-0.2524,  0.2291, -0.2641,  ..., -0.0915,  0.2037,  0.0492]],
       grad_fn=<AddmmBackward0>)


In [50]:
import torch
import torch.nn as nn

class Encoder(nn.Module):
    def __init__(self, input_vocab_size, embedding_dim, enc_units):
        """
        Args:
            input_vocab_size (int): Taille du vocabulaire en entrée.
            embedding_dim (int): Dimension de l'espace d'embedding.
            enc_units (int): Nombre d'unités dans la couche GRU bidirectionnelle.
        """
        super(Encoder, self).__init__()
        self.enc_units = enc_units
        self.input_vocab_size = input_vocab_size

        # La couche d'embedding convertit les tokens en vecteurs.
        self.embedding = nn.Embedding(input_vocab_size, embedding_dim)

        # La couche GRU bidirectionnelle.
        self.bigru = nn.GRU(embedding_dim, enc_units, bidirectional=True, batch_first=True)

    def forward(self, tokens, state=None):
        """
        Args:
            tokens (torch.Tensor): Séquence de tokens en entrée.
            state (torch.Tensor, optional): État initial de la couche GRU. Par défaut, None.

        Returns:
            torch.Tensor: Séquence encodée.
            torch.Tensor: État final de la couche GRU dans la direction arrière.
        """
        # Embedding des tokens d'entrée.
        vectors = self.embedding(tokens)

        # La couche GRU bidirectionnelle traite la séquence d'embedding.
        # La sortie a une forme de (batch, s, enc_units * 2)
        # L'état a une forme de (num_layers * num_directions, batch, enc_units)
        output, state = self.bigru(vectors, state)

        # La sortie contient à la fois l'information des directions avant et arrière.
        # Nous prenons seulement la partie correspondante à la direction arrière.
        # Le slicing [:, :, self.enc_units:] prend les dimensions correspondant à la direction arrière.
        backward_output = output[:, :, self.enc_units:]

        # Nous renvoyons la séquence encodée et l'état final de la direction arrière.
        return backward_output, state

import torch.nn.utils.rnn as rnn_utils

# Exemple d'utilisation
input_vocab_size = 1000
embedding_dim = 128
enc_units = 256

encoder = Encoder(input_vocab_size, embedding_dim, enc_units)

# Supposons que tokens_batch soit une liste de séquences d'indices de mots.
tokens_batch = [
    torch.randint(0, input_vocab_size, (5,)),   # Longueur 5
    torch.randint(0, input_vocab_size, (8,)),   # Longueur 8
    torch.randint(0, input_vocab_size, (6,)),   # Longueur 6
]

# Ajout du padding
padded_tokens_batch = rnn_utils.pad_sequence(tokens_batch, batch_first=True, padding_value=0)

#print(padded_tokens_batch)

# Utilisation de l'encodeur
output, final_state = encoder(padded_tokens_batch)

print(output, final_state )



tensor([[[-0.1419, -0.3834, -0.0322,  ..., -0.0372, -0.2247, -0.0284],
         [-0.2926, -0.0300, -0.3833,  ..., -0.0826,  0.2095, -0.1092],
         [-0.0649,  0.0589, -0.2137,  ..., -0.0094,  0.4779, -0.0252],
         ...,
         [-0.4876, -0.1592,  0.2788,  ...,  0.4932, -0.1656,  0.1080],
         [-0.4504, -0.1160,  0.2697,  ...,  0.4124, -0.1343,  0.0948],
         [-0.3177, -0.0596,  0.2093,  ...,  0.2608, -0.0819,  0.0626]],

        [[ 0.2601,  0.1709,  0.0422,  ..., -0.5095,  0.2808, -0.0230],
         [ 0.2512, -0.1345, -0.0254,  ..., -0.2761, -0.0472,  0.1546],
         [ 0.1078,  0.0720, -0.3357,  ..., -0.1189, -0.0094, -0.0388],
         ...,
         [ 0.0559,  0.0588, -0.3089,  ..., -0.1702,  0.0875,  0.0595],
         [-0.0696,  0.1379, -0.0028,  ..., -0.2555,  0.4171, -0.0326],
         [ 0.3129,  0.0525,  0.3948,  ..., -0.2594,  0.1489,  0.2302]],

        [[-0.2218,  0.3162, -0.1397,  ..., -0.4617,  0.0726, -0.1840],
         [-0.0944,  0.1677,  0.1523,  ..., -0

In [123]:
class BahdanauAttention(nn.Module):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.Wq = nn.Linear(units, units)
        self.Wk = nn.Linear(units, units)
        self.V = nn.Linear(units, 1, bias=False)

    def forward(self, query, memory):
        # Étapes 1 et 2 : Calcul des scores d'attention
        score = self.V(torch.tanh(self.Wq(query) + self.Wk(memory)))

        # Étape 3 : Calcul des poids d'attention
        attention_weights = torch.softmax(score, dim=1)

        # Étape 4 : Calcul du vecteur de contexte
        context_vector = torch.bmm(attention_weights.transpose(1, 2), memory)

        return context_vector, attention_weights

In [124]:
import torch
import torch.nn as nn

class Decoder(nn.Module):
    def __init__(self, output_vocab_size, embedding_dim, dec_units):
        super(Decoder, self).__init__()
        self.dec_units = dec_units
        self.output_vocab_size = output_vocab_size
        self.embedding_dim = embedding_dim

        # Étape 1 : La couche d'embedding convertit les IDs de token en vecteurs
        self.embedding = nn.Embedding(self.output_vocab_size, embedding_dim)

        # Étape 2 : Le GRU garde une trace de ce qui a été généré jusqu'à présent.
        self.gru = nn.GRU(embedding_dim, dec_units, batch_first=True)

        # Étape 3 : La sortie du GRU sera la requête pour la couche d'attention.
        self.attention = BahdanauAttention(dec_units)

        # Étape 4 : Eqn. (3) : conversion de `ct` à `at`
        self.Wc = nn.Linear(dec_units, dec_units, bias=False)

        # Étape 5 : Cette couche entièrement connectée produit les logits pour chaque
        # token de sortie.
        self.fc = nn.Linear(dec_units, self.output_vocab_size)

    def forward(self, x, hidden, enc_output):
        # Étape 1 : Embedding
        x = self.embedding(x)

        # Étape 2 : GRU
        output, hidden = self.gru(x, hidden)

        # Étape 3 : Attention
        context_vector, attention_weights = self.attention(output, enc_output)

        # Étape 4 : Eqn. (3) : conversion de `ct` à `at`
        context_vector = torch.tanh(self.Wc(context_vector))

        # Étape 5 : Logits
        logits = self.fc(context_vector)

        return logits, hidden, attention_weights
    
# Paramètres arbitraires pour le test
output_vocab_size = 1000
embedding_dim = 256
dec_units = 512
batch_size = 5
seq_len = 20

# Instanciation du décodeur
decoder = Decoder(output_vocab_size, embedding_dim, dec_units)

# Tenseurs d'entrée fictifs
input_sequence = torch.randint(0, output_vocab_size, (batch_size, seq_len))
hidden_state = torch.zeros((1, batch_size, dec_units))
encoder_output = torch.randn(batch_size, seq_len, dec_units)

# Passage avant (forward pass)
logits, new_hidden, attention_weights = decoder(input_sequence, hidden_state, encoder_output)

# Affichage des résultats
print("Logits shape:", logits.shape)
print("New hidden state shape:", new_hidden.shape)
print("Attention weights shape:", attention_weights.shape)

Logits shape: torch.Size([5, 1, 1000])
New hidden state shape: torch.Size([1, 5, 512])
Attention weights shape: torch.Size([5, 20, 1])


In [132]:
# Initialisation des dimensions fictives pour le test
batch_size = 32
seq_len = 10
embedding_dim = 256
dec_units = 512
output_vocab_size = 1000

# Création d'une instance de la classe Encoder
encoder = Encoder(input_vocab_size=output_vocab_size, embedding_dim=embedding_dim, enc_units=dec_units)

# Création d'une instance de la classe Decoder
decoder = Decoder(output_vocab_size=output_vocab_size, embedding_dim=embedding_dim, dec_units=dec_units)

# Tenseurs d'entrée fictifs pour l'encodeur
input_sequence = torch.randint(0, 1000, (batch_size, seq_len))

# Passage avant (forward pass) à travers l'encodeur
enc_output, enc_state = encoder(input_sequence)

# Affichage des résultats de l'encodeur
print("Encoder Output shape:", enc_output.shape)
print("Encoder State shape:", enc_state.shape)

# Tenseurs d'entrée fictifs pour le décodeur
input_sequence_decoder = torch.randint(0, 1000, (batch_size, seq_len))
hidden_state_decoder = torch.zeros((1, batch_size, dec_units))

# Passage avant (forward pass) à travers le décodeur
logits_decoder, new_hidden_decoder, attention_weights_decoder = decoder(
    input_sequence_decoder, hidden_state_decoder, enc_output)

# Affichage des résultats du décodeur
print("Logits shape (from decoder):", logits_decoder.shape)
print("New hidden state shape (from decoder):", new_hidden_decoder.shape)
print("Attention weights shape (from decoder):", attention_weights_decoder.shape)


TypeError: Encoder.__init__() got an unexpected keyword argument 'input_vocab_size'

In [44]:
from models.rnn import RNN

encoder_rnn = RNN(len(tokenized_most_frequent_english_words),100,2,"cuda" if torch.cuda.is_available() else "cpu",torch.nn.Tanh())

In [45]:
for x in tokenized_train_data:
    tensorized = torch.tensor(x["ids_en"]).to("cuda" if torch.cuda.is_available() else "cpu")
    result = encoder_rnn(to_vector(tensorized, len(tokenized_most_frequent_english_words)).to("cuda" if torch.cuda.is_available() else "cpu").view(1,-1,len(tokenized_most_frequent_english_words)))
    print(result[0].shape)
    break
print (encoder_rnn)

torch.Size([1, 4, 100])
RNN(
  (rnn): RNN(30001, 100, num_layers=2, batch_first=True)
)


BLEU SCORE

1-EXISTANT NLTK

In [62]:
import nltk 
hypothesis=['Le','chat','se','trouve','sur','le','toit']
reference=['Le','chat','se','trouve','sur','le','toit','bla']
BLEUscore = nltk.translate.bleu_score.sentence_bleu([reference], hypothesis) 
print(BLEUscore)


0.8668778997501817


2-SCORE PERSONNALISE

In [100]:
import collections
import math

def ngram_precision(candidate, reference, n):
    candidate_ngrams = [tuple(candidate[i:i+n]) for i in range(len(candidate)-n+1)]
    reference_ngrams = [tuple(reference[i:i+n]) for i in range(len(reference)-n+1)]

    candidate_ngram_counts = collections.Counter(candidate_ngrams)
    reference_ngram_counts = collections.Counter(reference_ngrams)

    overlap_count = sum((candidate_ngram_counts & reference_ngram_counts).values())
    total_count = max(1, len(candidate_ngrams))

    precision = overlap_count / total_count
    return precision

def bleu_score(candidate, references, weights):
    precisions = []

    for i in range(len(weights)):
        n = i + 1
        precision = ngram_precision(candidate, references, n)
        precisions.append(precision)

    geometric_mean = math.exp(sum(weights[i] * math.log(p) for i, p in enumerate(precisions)))

    brevity_penalty = min(1, len(candidate) / max(len(reference) for reference in references))

    bleu = brevity_penalty * geometric_mean
    return bleu

# Exemple d'utilisation
reference = ['the', 'quick', 'brown', 'fox', 'jumped', 'over', 'the', 'lazy', 'dog']
candidate = ['the', 'fast', 'brown', 'fox', 'jumped', 'over', 'the', 'lazy', 'dog']

candidate=['Le','chat','se','trou','s','le','to','bli']
reference=['Le','chat','se','trouve','sur','le','toit']

weights = [0.25,0.25,0.25] # You can adjust the weights based on your preference

score = bleu_score(candidate, reference, weights)
print("BLEU Score:", score)


BLEU Score: 0.392814650900513
